## Installs for new environments

In [4]:
# import sys
# !{sys.executable} -m pip install tensorflow
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install pillow
# !{sys.executable} -m pip install scipy

In [9]:
%load_ext tensorboard

## Imports

In [13]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import math

import datetime


## Config

In [24]:
image_dim = 256 # eventual width/height of images
input_shape = (image_dim, image_dim, 3) #(width, height, RGB)

epochs = 8
batch_size = 32
learning_rate = .001

train_dir = 'dataset/train'
validate_dir = 'dataset/validate'

save_dir = 'models/reinforced_full_val_correct'
file_name_base = 'reinforced_full_val_correct'

base_log_dir = '/logs'

## Augmentation Definition

In [18]:
train = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True) # Reinforcements for training dataset

validate = ImageDataGenerator(rescale=1./255) # Reinforcements for validation dataset

## Dataset Loading

In [19]:
#loads the datasets
train_dataset = train.flow_from_directory(train_dir, target_size=(image_dim, image_dim), class_mode='categorical') 
validate_dataset = validate.flow_from_directory(validate_dir, target_size=(image_dim, image_dim), class_mode='categorical')

# tf.keras.preprocessing.image_dataset_from_directory - this can be used for no reinforcement

n_train = train_dataset.samples # number of images in training dataset
n_validate = validate_dataset.samples # number of images in validation dataset


Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


## Dataset Loading: No Augmentation

In [15]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, image_size=(image_dim, image_dim))
validate_dataset = tf.keras.utils.image_dataset_from_directory(validate_dir, image_size=(image_dim, image_dim), class_mode='categorical')

n_train = train_dataset.samples # number of images in training dataset
n_validate = validate_dataset.samples # number of images in validation dataset

TypeError: Unknown keywords argument(s): ('target_size', 'class_mode')

## CNN Design

In [22]:
class_keys = list(train_dataset.class_indices.keys())
n_classes = len(class_keys)

# defines the layers of the cnn
model = tf.keras.models.Sequential([
    Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(n_classes, activation='softmax')
])

## Compilation

In [23]:
log_dir = base_log_dir + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Evaluate and Save the initial model

In [25]:
model.evaluate(validate_dataset, batch_size=256)
model.save(save_dir + '/' + file_name_base + '_0')

63/63 [==============================] - 60s 946ms/step - loss: 2.3038 - accuracy: 0.1010
INFO:tensorflow:Assets written to: models/reinforced_full_val_correct/reinforced_full_val_correct_0\assets


## Training

In [11]:
model = tf.keras.models.load_model(save_dir + '/' + file_name_base + '_30')

In [26]:
for i in range(0,4):
    print((i+1) * epochs)
    model.fit(train_dataset,
                        steps_per_epoch=math.floor(n_train/batch_size),
                        validation_data=validate_dataset,
                        validation_steps=math.floor(n_validate/batch_size),
                        epochs=epochs,
                        callbacks=[tensorboard])
    model.save(save_dir + '/' + file_name_base + '_' + str((i+1)*epochs))


8
Epoch 1/8
250/250 [==============================] - 909s 4s/step - loss: 2.2943 - accuracy: 0.1265 - val_loss: 2.2682 - val_accuracy: 0.1618
Epoch 2/8
 48/250 [====>.........................] - ETA: 11:35 - loss: 2.2571 - accuracy: 0.1497

## Save Trained Model

In [19]:
model.save(save_dir + '/' + file_name_base + '_45')

INFO:tensorflow:Assets written to: models/reinforced_full/reinforced_full_45\assets
